In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt

import torch

%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
# import torch.utils.data.DataLoader 

#train and test data directory
data_dir = "/content/drive/MyDrive/archive/tire-dataset/"
test_data_dir = "/content/drive/MyDrive/archive/tire-test-dataset/"


#load the train and test data
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
# dataset=dataset.to("cuda")
test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
test_dataset1 = ImageFolder("/content/drive/MyDrive/Testing/",transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
# oneimage= DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
#            batch_sampler=None, num_workers=0, collate_fn=None,
#            pin_memory=False, drop_last=False, timeout=0,
#            worker_init_fn=None, *, prefetch_factor=2,
#            persistent_workers=False)
# test_dataset=test_dataset.to(device)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    def validation_step_for_one(self,batch,id):
        images,labels=batch
        out=self(images)
        return {"Prediction":out[id],"realanwswer":labels[id]}
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

val_dl1 = DataLoader(test_dataset1, 128,shuffle=False, num_workers = 2, pin_memory = True)

In [ ]:
class TireClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,3)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [ ]:

img, label = test_dataset[0]
# print(img,label)

In [ ]:
model = torch.load('/content/drive/MyDrive/archive/model_file')
model.eval()

TireClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=82944, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_

In [ ]:
# model.load_state_dict(torch.load(PATH))
# # model.eval()
# print(model.state_dict())

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/archive/Param_file"))
model.eval()

TireClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=82944, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_

In [ ]:

print(model.state_dict())


OrderedDict([('network.0.weight', tensor([[[[ 0.1298,  0.0028,  0.0912],
          [-0.1906,  0.0821,  0.1389],
          [-0.0232,  0.1764,  0.1607]],

         [[-0.1080, -0.0869, -0.1758],
          [-0.0333,  0.0824,  0.0708],
          [-0.1273, -0.1710, -0.0702]],

         [[-0.1356, -0.0213, -0.1986],
          [ 0.0512,  0.0095, -0.0655],
          [ 0.0461,  0.0723,  0.0333]]],


        [[[-0.1349,  0.0037,  0.1521],
          [-0.0805, -0.1477,  0.1098],
          [ 0.0590,  0.0318,  0.0024]],

         [[ 0.0543,  0.0986, -0.0695],
          [ 0.0457, -0.0119,  0.1329],
          [ 0.1165,  0.0833, -0.1273]],

         [[ 0.1619,  0.0789,  0.0682],
          [-0.0018, -0.0043, -0.0352],
          [ 0.1105,  0.1671, -0.0276]]],


        [[[-0.1169, -0.0781,  0.0646],
          [-0.1358,  0.1607,  0.0778],
          [-0.0057, -0.0483,  0.1339]],

         [[-0.0661,  0.1574, -0.0296],
          [ 0.1043,  0.0795, -0.0459],
          [-0.0246,  0.1027,  0.1311]],

         [

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

@torch.no_grad()
def evaluate_for_one(model,val_loader,id,id_b):
    model.eval()
    xer=0
    for batch in val_loader:
        if xer==id_b:
            output = model.validation_step_for_one(batch,id) 
            return output
        xer+=1
    return "invalid id_b"

    
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            # batch = batch.to(device)
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        print(result)
    
    return history

In [ ]:
tester = evaluate_for_one(model,val_dl1,4,0)
# tester1 = evaluate_for_one(model,val_dl1,1,0)
# tester2 = evaluate_for_one(model,val_dl1,2,0)
# tester3 = evaluate_for_one(model,val_dl1,4,0)
# tester4 = evaluate_for_one(model,val_dl1,5,0)
# tester5 = evaluate_for_one(model,val_dl1,6,0)
# tester6 = evaluate_for_one(model,val_dl1,7,0)
# tester7 = evaluate_for_one(model,val_dl1,8,0)

In [ ]:
print(tester)
# print(tester1)
# print(tester2)
# print(tester3)
# print(tester4)
# print(tester5)
# print(tester6)
# print(tester7)

{'Prediction': tensor([ -4.1102, -20.7502,  15.9918]), 'realanwswer': tensor(2)}


In [ ]:

tester = evaluate_for_one(model,val_dl1,0,0)
print(tester)
tester = evaluate_for_one(model,val_dl1,1,0)
print(tester)
tester = evaluate_for_one(model,val_dl1,2,0)
print(tester)
tester = evaluate_for_one(model,val_dl1,3,0)
print(tester)
tester = evaluate_for_one(model,val_dl1,4,0)
print(tester)

{'Prediction': tensor([ 18.0031,  -2.4856, -23.7478]), 'realanwswer': tensor(0)}
{'Prediction': tensor([-12.6368, -16.1446,  27.6716]), 'realanwswer': tensor(0)}
{'Prediction': tensor([-42.9483, -25.8021,  70.4717]), 'realanwswer': tensor(1)}
{'Prediction': tensor([-20.0576,  -3.3477,  20.7209]), 'realanwswer': tensor(2)}
{'Prediction': tensor([ -4.1102, -20.7502,  15.9918]), 'realanwswer': tensor(2)}


In [1]:
# from torch.nn.modules.linear import Identity
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

def show_batch(dl,id):
        """Plot images grid of single batch"""
        xer=0
        for images, labels in dl:
            # images[id].
              fig,ax = plt.subplots(figsize = (16,12))
              ax.set_xticks([])
              ax.set_yticks([])
              print(labels[id])
              ax.imshow(images[id].permute(1,2,0))
              break
            
show_batch(val_dl1,4)

NameError: ignored

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        print(labels)
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(val_dl1)
# train_dl.shape()